In [1]:
#python_bitbankccのパッケージをインポート
import python_bitbankcc 
import datetime
import os 
import time
import numpy as np
import pandas as pd
import sys
#機械学習用のモジュール
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from sklearn import linear_model
% matplotlib inline
from __future__ import print_function
import copy
import matplotlib
matplotlib.style.use('ggplot')

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# public API classのオブジェクトを取得
pub = python_bitbankcc.public()

In [3]:
#APIから基本データの取得、dataframeへ挿入
def get_candle(trade_name,span,back_day):
    # ロウソク足データを取得
    value = pub.get_candlestick( trade_name,span, back_day )
    candle = value['candlestick'][0]
    #print(candle['ohlcv'][0])
    dataframe = pd.DataFrame(candle['ohlcv'],columns=["Open", "High","Low","Close","Volume","Timestamp"])
    return dataframe


In [4]:
#指定した日から今日までの基本データをdataframeにして取得
#back_day,todayはdatetime
# span = ['1min', '5min', '15min', '30min', '1hour', '4hour', '8hour', '12hour', '1day', '1week']
#trade_name = ['btc_jpy', 'xrp_jpy', 'ltc_btc', 'eth_btc', 'mona_jpy', 'mona_btc', 'bcc_jpy', 'bcc_btc']

def make_df(trade_name,span,back_day,today):
    i = 1
    #基準が９時なので、そこを合わせてあげる
    if 0 <= today.hour < 9:
        k = 1
    elif 9 <= today.hour <= 23:
        k = 0
        
    while back_day <= today - datetime.timedelta(days=k):
        if i == 1:
            df1 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y%m%d'))
            back_day = back_day + datetime.timedelta(days=1)
            print(back_day,len(df1))
            i += 1
        else:
            df2 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y%m%d'))
            df1 = pd.concat([df1, df2])
            back_day = back_day + datetime.timedelta(days=1)
            print(back_day,len(df1))
            i += 1
    
    return df1


In [5]:
def read_date(x):
    return datetime.datetime.fromtimestamp(x/1000)


In [6]:
#関数の使い方
today = datetime.datetime.today() 

back_day = today - datetime.timedelta(days=300)
trade_name = 'xrp_jpy'
span = '12hour'

In [7]:
#df1 =  make_df(trade_name,span,back_day,today)
df1 = get_candle( 'xrp_jpy',  '12hour', '2017')
df2 = get_candle( 'xrp_jpy',  '12hour', '2018')
df1 = pd.concat([df1, df2])
df1['Timestamp'] = df1['Timestamp'].apply(read_date)
#indexの割り振り
df1 = df1.reset_index()
#df1の値は全てstrなのでTimestamp以外floatにキャストする
#Timestampは文字列のままでいいので削除することを指定
execlude = ['Timestamp']
cast = [col for col in df1.columns if col not in execlude]
for item in cast:
    df1 = df1.astype({item: float})
print(df1)

     index    Open    High     Low   Close        Volume           Timestamp
0      0.0  34.375  34.375  34.375  34.375  0.000000e+00 2017-05-24 21:00:00
1      1.0  34.375  35.500  34.000  34.000  3.072000e+03 2017-05-25 09:00:00
2      2.0  34.000  36.000  31.000  35.000  6.133000e+03 2017-05-25 21:00:00
3      3.0  35.000  40.000  30.200  40.000  9.901000e+03 2017-05-26 09:00:00
4      4.0  40.000  40.000  30.500  36.000  1.254100e+04 2017-05-26 21:00:00
5      5.0  36.000  36.000  20.000  20.000  9.026198e+04 2017-05-27 09:00:00
6      6.0  20.000  36.000  20.000  27.000  4.969786e+03 2017-05-27 21:00:00
7      7.0  27.000  29.000  27.000  29.000  1.358398e+04 2017-05-28 09:00:00
8      8.0  29.000  29.000  20.500  29.000  7.202677e+03 2017-05-28 21:00:00
9      9.0  29.000  29.550  25.500  25.500  1.124225e+04 2017-05-29 09:00:00
10    10.0  25.500  27.900  25.500  25.501  1.260000e+03 2017-05-29 21:00:00
11    11.0  25.501  27.500  25.501  27.000  4.440000e+02 2017-05-30 09:00:00

In [8]:

# 単純移動平均（SMA）を取得する関数
#上記のdfデータフレーム
#numいくつの平均を求めるか
def getMA( df,timeframe,num ):
 
    tmp = []
    avg = np.array([])
    for i in range(len(df) - num + 1):
        for j in range(num):
            tmp.append( df['Close'][i+j])
         # 平均値計算
        value = np.average(tmp)
        avg = np.append(avg,value)
        tmp = []
    return avg

In [9]:
MA5 = getMA( df1,'1hour' ,5)
MA25 = getMA( df1,'1hour' ,25)

In [10]:
print(MA5.shape,MA25.shape)
print(type(MA5[1]))

(986,) (966,)
<class 'numpy.float64'>


In [11]:
zero = np.zeros(20,dtype = float )
print(zero)
MA25 = np.insert(MA25, 0, zero)
print(MA25,MA25.shape,type(MA25))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[  0.        0.        0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.       31.52784
  31.42044  31.32044  31.16844  30.80844  30.65244  31.1244   31.3256
  31.4376   31.46964  31.62964  31.76564  31.8856   31.8936   32.11352
  32.14444  31.94444  31.66052  31.43948  31.12988  30.91304  30.79704
  30.61112  30.34392  30.24092  30.0942   30.007    29.951    29.86852
  30.0346   30.1906   30.23464  30.17344  30.12584  30.18536  30.25736
  30.40532  30.52576  30.67296  30.66064  30.69772  30.69372  30.6288
  30.54852  30.59044  30.66088  30.75088  30.7556   30.7356   30.72924
  30.72724  30.68944  30.59744  30.53968  30.2696   29.9546   29.7782
  29.6982   29.6132   29.49364  29.36644  29.19196  29.00432  28.82152
  28.63668  28.43748  28.27624  28.17692  28.06452  27.9008   27.59244
  27.2314   26.89264  26.61 

In [12]:
result = pd.DataFrame(MA5)
result.columns = ['MA5']
result['MA25'] = MA25
#２乗平均誤差
print(result.loc[:])

         MA5      MA25
0    35.8750   0.00000
1    33.0000   0.00000
2    31.6000   0.00000
3    30.4000   0.00000
4    28.2000   0.00000
5    26.1000   0.00000
6    27.2002   0.00000
7    27.2002   0.00000
8    26.9802   0.00000
9    26.3802   0.00000
10   27.0802   0.00000
11   28.9800   0.00000
12   31.1400   0.00000
13   32.8566   0.00000
14   34.9366   0.00000
15   35.7366   0.00000
16   34.9366   0.00000
17   33.9764   0.00000
18   33.8398   0.00000
19   33.0038   0.00000
20   32.8474  31.52784
21   32.9854  31.42044
22   32.6856  31.32044
23   31.7656  31.16844
24   31.5216  30.80844
25   31.4980  30.65244
26   31.5198  31.12440
27   31.6258  31.32560
28   31.7458  31.43760
29   31.5060  31.46964
..       ...       ...
956  31.2558  31.30864
957  31.3480  31.08888
958  31.1460  30.94728
959  31.8002  31.09608
960  32.7000  31.25712
961  33.5024  31.34020
962  34.4560  31.50276
963  36.5108  31.84612
964  39.6508  32.56764
965  44.3990  33.71284
966  49.5162  34.88836
967  54.877

In [13]:
# Plot
plt.plot(range(0,len(result)), MA5, label='MA5', color='blue')
plt.plot(range(0,len(result)), MA25, label='MA25', color='red')
plt.xlabel('time')
plt.ylabel('Price(XRP)')
plt.title('XRP Price')
plt.grid(True)
plt.legend()
plt.savefig('XRP_MA.png')
plt.close()

In [14]:
#なし、ゴールデンクロス、デッドクロス判定の「０、１」判定配列作成
#正解かどうかを可視化する ゴールデン・デッドクロス判定
def golden_dead(data_df1):
    golden=np.zeros(len(data_df1['Cross']),dtype=int)
    dead=np.zeros(len(data_df1['Cross']),dtype=int)
    for i in range(len(data_df1['Cross'])-1):
        x = data_df1['Cross'][i]
        y = data_df1['Cross'][i+1]
        if ((x <= 0) & (y<=0)) |((x >= 0) & (y>=0)):
            pass
        elif ((x <= 0) & (y>=0)) :
            golden=np.insert(golden,i+1,1)
            golden=np.delete(golden,i+2)
        elif ((x >= 0) & (y<=0)) :
            dead=np.insert(dead,i+1,1)
            dead=np.delete(dead,i+2)

    return golden,dead
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-14-3deb5ca1a110>, line 5)